## 🌐 Run in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ayansh1729/Data-generation/blob/main/notebooks/01_getting_started.ipynb)

**Running in Google Colab?** Execute the cell below to set up everything automatically!


In [ ]:
# 🚀 GOOGLE COLAB SETUP - Run this cell first if using Colab!
import os
import sys
import subprocess

# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Running in Google Colab")
except:
    IN_COLAB = False
    print("💻 Running locally")

if IN_COLAB:
    print("\n" + "="*70)
    print("🔧 Setting up environment for Google Colab...")
    print("="*70)
    
    # Clone the repository
    print("\n📥 Step 1/4: Cloning repository...")
    if not os.path.exists('/content/Data-generation'):
        subprocess.run(['git', 'clone', 'https://github.com/ayansh1729/Data-generation.git'], 
                      cwd='/content', check=True)
        print("   ✅ Repository cloned")
    else:
        print("   ✅ Repository already exists")
    
    # Change to project directory
    os.chdir('/content/Data-generation')
    print(f"   📂 Working directory: {os.getcwd()}")
    
    # Install dependencies
    print("\n📦 Step 2/4: Installing dependencies...")
    print("   (This may take 2-3 minutes...)")
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-r', 'requirements.txt'], 
                   check=True)
    print("   ✅ Dependencies installed")
    
    # Install the package
    print("\n⚙️  Step 3/4: Installing project as package...")
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-e', '.'], 
                   check=True)
    print("   ✅ Project installed")
    
    # Add to Python path
    sys.path.insert(0, '/content/Data-generation')
    
    # Verify installation
    print("\n🧪 Step 4/4: Verifying installation...")
    try:
        from src.models.diffusion import DDPM
        from src.data.dataset import create_dataloader
        from src.explainability.gradcam import GradCAMExplainer
        print("   ✅ All modules imported successfully!")
    except ImportError as e:
        print(f"   ⚠️  Import warning: {e}")
    
    # Check GPU
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"\n🎮 GPU Available: {gpu_name}")
        print(f"   Memory: {gpu_memory:.1f} GB")
    else:
        print("\n⚠️  No GPU detected. Training will be slower.")
        print("   💡 Enable GPU: Runtime → Change runtime type → GPU")
    
    print("\n" + "="*70)
    print("✅ Setup complete! You can now run the rest of the notebook.")
    print("="*70)
    
else:
    print("ℹ️  Not in Colab - skipping Colab setup")
    print("   If running locally, make sure you've run: pip install -e .")


In [ ]:
# Verify all key packages are installed
import sys

def check_package(package_name, import_name=None):
    """Check if a package is installed"""
    if import_name is None:
        import_name = package_name
    try:
        module = __import__(import_name)
        version = getattr(module, '__version__', 'unknown')
        print(f"✅ {package_name:20s} - version {version}")
        return True
    except ImportError:
        print(f"❌ {package_name:20s} - NOT INSTALLED")
        return False

print("📋 Checking installed packages...\n")
print("=" * 60)

packages = [
    ('torch', 'torch'),
    ('torchvision', 'torchvision'),
    ('numpy', 'numpy'),
    ('matplotlib', 'matplotlib'),
    ('seaborn', 'seaborn'),
    ('pandas', 'pandas'),
    ('sklearn', 'sklearn'),
    ('diffusers', 'diffusers'),
    ('tqdm', 'tqdm'),
    ('PIL', 'PIL'),
    ('yaml', 'yaml'),
    ('omegaconf', 'omegaconf'),
]

all_installed = all(check_package(name, import_name) for name, import_name in packages)

print("=" * 60)

if all_installed:
    print("\n🎉 All core packages are installed!")
    print("✅ You're ready to go!")
else:
    print("\n⚠️  Some packages are missing. Please run:")
    print("   pip install -r requirements.txt")

# Check CUDA availability
import torch
print(f"\n🔧 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("   Running on CPU (slower but still works)")


In [ ]:
# Core libraries
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Framework components
from src.models.diffusion import DDPM
from src.models.unet import UNet
from src.data.dataset import create_dataloader
from src.training.losses import get_loss_function
from src.training.scheduler import get_scheduler
from src.explainability.gradcam import GradCAMExplainer, plot_gradcam_grid
from src.explainability.attention_viz import AttentionVisualizer
from src.explainability.diffusion_trace import DiffusionTracer
from src.utils.visualization import plot_samples, plot_training_curves, plot_comparison
from src.utils.config import load_config

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Using device: {device}")
print(f"🐍 PyTorch version: {torch.__version__}")
print(f"✅ Setup complete!")


In [ ]:
# Create dataloader for CIFAR-10
train_loader = create_dataloader(
    dataset_name='cifar10',
    batch_size=64,
    num_workers=2,
    image_size=32,
    split='train'
)

print(f"📦 Dataset loaded!")
print(f"   - Number of batches: {len(train_loader)}")
print(f"   - Batch size: {train_loader.batch_size}")
print(f"   - Total images: ~{len(train_loader) * train_loader.batch_size:,}")

# Get a batch of real images
real_batch = next(iter(train_loader))
print(f"\n🖼️  Batch shape: {real_batch.shape}")
print(f"   - Range: [{real_batch.min():.2f}, {real_batch.max():.2f}]")


In [ ]:
# Visualize real samples
fig = plot_samples(real_batch[:64], nrow=8, title="Real CIFAR-10 Images", figsize=(14, 14))
plt.show()


In [ ]:
# Model configuration
model_config = {
    'image_size': 32,
    'channels': 3,
    'timesteps': 1000,
    'noise_schedule': 'cosine',
    'dim': 64,
    'dim_mults': (1, 2, 4, 8),
    'num_res_blocks': 2,
    'attention_resolutions': (16, 8),
    'dropout': 0.1
}

# Create the diffusion model
model = DDPM(**model_config).to(device)

# Count parameters
num_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"🏗️  Model Architecture:")
print(f"   - Total parameters: {num_params:,}")
print(f"   - Trainable parameters: {trainable_params:,}")
print(f"   - Model size: ~{num_params * 4 / 1024 / 1024:.2f} MB")
print(f"\n✅ Model created successfully!")


In [ ]:
# Training configuration
training_config = {
    'num_epochs': 2,
    'learning_rate': 2e-4,
    'weight_decay': 0.0,
    'gradient_clip': 1.0,
}

# Setup optimizer and scheduler
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=training_config['learning_rate'],
    weight_decay=training_config['weight_decay']
)

scheduler = get_scheduler(
    optimizer,
    scheduler_type='cosine',
    T_max=training_config['num_epochs'] * len(train_loader),
    eta_min=1e-6
)

loss_fn = get_loss_function('mse')

print(f"⚙️  Training Configuration:")
print(f"   - Epochs: {training_config['num_epochs']}")
print(f"   - Learning rate: {training_config['learning_rate']}")
print(f"   - Optimizer: AdamW")
print(f"   - Scheduler: Cosine Annealing")
print(f"   - Gradient clip: {training_config['gradient_clip']}")
print(f"\n✅ Training setup complete!")


In [ ]:
# Training loop
model.train()
losses = []
best_loss = float('inf')

for epoch in range(training_config['num_epochs']):
    epoch_losses = []
    
    pbar = tqdm(train_loader, desc=f"📈 Epoch {epoch+1}/{training_config['num_epochs']}")
    for batch_idx, batch in enumerate(pbar):
        batch = batch.to(device)
        
        # Forward pass
        outputs = model(batch)
        loss = outputs['loss']
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(
            model.parameters(),
            training_config['gradient_clip']
        )
        
        optimizer.step()
        scheduler.step()
        
        epoch_losses.append(loss.item())
        pbar.set_postfix({'loss': f"{loss.item():.4f}", 'lr': f"{scheduler.get_last_lr()[0]:.2e}"})
        
        # Early stop for demo (train on 100 batches only)
        if batch_idx >= 100:
            break
    
    avg_loss = np.mean(epoch_losses)
    losses.append(avg_loss)
    
    print(f"✨ Epoch {epoch+1} complete - Average Loss: {avg_loss:.4f}")
    
    if avg_loss < best_loss:
        best_loss = avg_loss
        print(f"   🎯 New best loss!")

print(f"\n🎉 Training complete!")


In [ ]:
# Plot training curve
fig = plot_training_curves(losses, title="Training Loss Curve", figsize=(10, 6))
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# Generate synthetic samples
model.eval()
with torch.no_grad():
    print("🎲 Generating synthetic samples...")
    print("   (This takes ~1-2 minutes for 1000 denoising steps)")
    
    generated_samples = model.sample(batch_size=64)
    
print(f"\n✅ Generated {generated_samples.shape[0]} synthetic images!")
print(f"   Shape: {generated_samples.shape}")
print(f"   Range: [{generated_samples.min():.2f}, {generated_samples.max():.2f}]")


In [ ]:
# Visualize generated samples
fig = plot_samples(
    generated_samples,
    nrow=8,
    title="🎨 Generated Synthetic Images",
    figsize=(14, 14)
)
plt.show()


In [ ]:
# Compare real vs generated
fig = plot_comparison(
    real_batch[:64],
    generated_samples[:64],
    nrow=8,
    figsize=(14, 7)
)
plt.tight_layout()
plt.show()


In [ ]:
# Create GradCAM explainer
explainer = GradCAMExplainer(model, target_layers=['mid_attn', 'mid_block1'])

# Select samples to explain
x_samples = real_batch[:4].to(device)
t_timesteps = torch.tensor([250, 500, 750, 999]).to(device)

print("🔬 Computing GradCAM explanations...")
print(f"   - Samples: {x_samples.shape[0]}")
print(f"   - Timesteps: {t_timesteps.tolist()}")
print(f"   - Target layers: {explainer.target_layers}")


In [ ]:
# Compute GradCAM at different timesteps
for idx, t in enumerate(t_timesteps):
    t_batch = torch.full((x_samples.shape[0],), t.item(), device=device, dtype=torch.long)
    
    gradcams = explainer.compute_gradcam(x_samples, t_batch)
    visualizations = explainer.visualize_gradcam(x_samples, t_batch, alpha=0.5)
    
    print(f"\n📊 Timestep t={t.item()}:")
    print(f"   - Computed GradCAM for {len(gradcams)} layers")
    
    # Visualize
    for layer_name, viz in list(visualizations.items())[:1]:  # Show first layer
        fig, axes = plt.subplots(1, 4, figsize=(12, 3))
        for i in range(4):
            axes[i].imshow(viz[i])
            axes[i].axis('off')
            axes[i].set_title(f'Sample {i+1}', fontsize=10)
        
        plt.suptitle(f'GradCAM at t={t.item()} - Layer: {layer_name}', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
    if idx >= 1:  # Show only 2 timesteps for demo
        break

# Cleanup
explainer.remove_hooks()
print("\n✅ GradCAM analysis complete!")
